# TensorFlow introduction

TensorFlow is an open source software library for numerical computation using data flow graphs. In a data flow graph, nodes represent mathematical operations and edges represent the multidimensional data arrays (tensors) communicated between them.

![](https://raw.githubusercontent.com/wdbm/abstraction/master/media/2016-05-14T1754Z.png)

TensorFlow is usually used in Python, though in the background it is using hardcore efficient code to parallelize its calculations a lot and is well-suited to GPU hardware. The Python convention to import TensorFlow is as follows:

In [1]:
import tensorflow as tf

It can be helpful to hide some TensorFlow logging messages:

In [2]:
tf.TF_CPP_MIN_LOG_LEVEL = 3

# tensors, ranks, shapes and types

The central unit of data in TensorFlow is the tensor, in the sense of it being an array of some arbitrary dimensionality. A tensor of rank 0 is a scalar, a tensor of rank 1 is a vector, a tensor of rank 2 is a matrix, a tensor of rank 3 is a 3-tensor, and so on.

|**rank**|**mathamatical object**|**shape**  |**example**                       |
|--------|-----------------------|-----------|----------------------------------|
|0       |scalar                 |`[]`       |`3`                               |
|1       |vector                 |`[3]`      |`[1. ,2., 3.]`                    |
|2       |matrix                 |`[2, 3]`   |`[[1., 2., 3.], [4., 5., 6.]]`    |
|3       |3-tensor               |`[2, 1, 3]`|`[[[1., 2., 3.]], [[7., 8., 9.]]]`|
|n       |n-tensor               |...        |...                               |

The various number types that TensorFlow can handle are as follows:

|**data type**|Python type|**description**       |
|-------------|-----------|----------------------|
|`DT_FLOAT`   |`t.float32`|32 bits floating point|
|`DT_DOUBLE`  |`t.float64`|64 bits floating point|
|`DT_INT8`    |`t.int8`   |8 bits signed integer |
|`DT_INT16`   |`t.int16`  |16 bits signed integer|
|`DT_INT32`   |`t.int32`  |32 bits signed integer|
|`DT_INT64`   |`t.int64`  |64 bits signed integer|

# TensorFlow mechanics: computational graphs and nodes

TensorFlow programs are defined as computational graphs. For TensorFlow, a computational graph is a series of TensorFlow operations arranged in a graph of nodes. A node takes zero or more tensors as inputs and produces a tensor as an output. Generally, a TensorFlow program consists of sections like these:

- 1 Build a graph using TensorFlow operations.
- 2 Feed data to TensorFlow and run the graph.
- 3 Update variables in the graph and return values.

A simple TensorFlow node is a constant. It takes no inputs and simply outputs a value that it stores internally. Here are some constants:

In [3]:
node_1 = tf.constant(3.0, tf.float32)
node_2 = tf.constant(4.0) # (also tf.float32 by default)

print("node_1: {node}".format(node = node_1))
print("node_2: {node}".format(node = node_2))

node_1: Tensor("Const:0", shape=(), dtype=float32)
node_2: Tensor("Const_1:0", shape=(), dtype=float32)


The printouts of the nodes do not evaluate the outputs the nodes would produce, but show simply what the nodes would evaluate. To evaluate nodes, the computational graph is run in an encapsulation of the control and state of the TensorFlow runtime called a TensorFlow "session".

# session

A `Session` is a class for running TensorFlow operations. A session object encapsulates the environment in which operations are executed and tensors are evaluated. For example, `sess.run(c)` evaluates the tensor `c`.

A session is run using its `run` method:

```Python
tf.Session.run(
    fetches,
    feed_dict    = None,
    options      = None,
    run_metadata = None
)
```

This method runs operations and evaluates tensors in fetches. It returns one epoch of TensorFlow computation, by running the necessary graph fragment to execute every operation and evaluate every tensor in fetches, substituting the values in `feed_dict` for the corresponding input values. The `fetches` option can be a single graph element, or an arbitrary nested list, tuple, namedtuple, dict or OrderedDict containing graph elements at its leaves. The value returned by `run` has the same shape as the fetches argument, where the leaves are replaced by the corresponding values returned by TensorFlow.

So, those constant nodes could be evaluated like this:

In [4]:
sess = tf.Session()

sess.run([node_1, node_2])

[3.0, 4.0]

More complicated nodes than constants are operations. For example, two constant nodes could be added:

In [5]:
node_3 = tf.add(node_1, node_2)

node_3

<tf.Tensor 'Add:0' shape=() dtype=float32>

In [6]:
sess.run(node_3)

7.0

This is, of course, a trivial mathematical operation, but it has been performed using very computationally efficient infrastructure. Far more complicated operations can be encoded in a computational graph and run using TensorFlow.

# placeholders

A computational graph can be parameterized to accept external inputs. These entry points for data are called placeholders.

So, let's create some placeholders that can hold 32 bit floating point numbers and let's also make a node for the addition operation applied to these placeholders:

In [7]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

adder_node

<tf.Tensor 'add:0' shape=<unknown> dtype=float32>

The `feed_dict` parameter of a session `run` method is used to input data to these placeholders:

In [8]:
print(sess.run(adder_node, {a: 3, b: 4}))

7.0


The same can be done with multiple values:

In [9]:
print(sess.run(adder_node, {a: [3, 4], b: [5, 6]}))

[  8.  10.]


You can start to see now how parallelism is core to TensorFlow.

Further nodes can be added to the computational graph easily:

In [10]:
add_and_triple = adder_node * 3.

sess.run(add_and_triple, {a: 3, b: 4})

21.0

# variables

Variables are nodes that have values that can change. These are used to have variable values in models, to make models trainable. A variable is defined with a type and an initial value.

We can make a linear model featuring changable variables like this:

In [11]:
W = tf.Variable([.3], dtype = tf.float32)
b = tf.Variable([- .3], dtype = tf.float32)
x = tf.placeholder(tf.float32)

linear_model = W * x + b

Constants are initialized when they are called and their value doesn't change, but variables are initialized in a TensorFlow program using a special operation:

In [12]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

Since `x` is a placeholder, this linear model can be evaluated for several `x` values in parallel:

In [13]:
print(sess.run(linear_model, {x:[1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


A large number of values can be stored in a variable easily, like this:

In [14]:
weights = tf.Variable(
    tf.random_normal(
        [784, 200],
        stddev = 0.35
    ),
    name = "weights"
)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

print(weights)

Tensor("weights/read:0", shape=(784, 200), dtype=float32)
